In [2]:
class Red_Neuronal():
    def __init__(self, num_neuronas_capa, f_act_capa):
        self.num_neuronas_capa = num_neuronas_capa
        self.f_act_capa = f_act_capa

In [ ]:
rn1 = Red_Neuronal([1,2,3], [1,1,1])